# An Implementation of Fast Agglomerative Hierarchical Clustering Algorithm Using Locality-Sensitive Hashing by Walker Harrison and Lisa Lebovici (2018)

## Abstract

Agglomerative hierarchical clustering is a widely used clustering algorithm in which n data points are sequentially merged according to some distance metric from n clusters into a single cluster. While many different metrics exist to evaluate inter-cluster distance, single-linkage reigns among the most popular; according to this method, pairwise distances are computed between all points that do not share a cluster, and the clusters containing the two points with the shortest euclidean distance are joined in each iteration. However, single-linkage has some drawbacks, most noteworthy that it scales quadratically as data grows. As such, Koga, Ishibashi, and Watanbe propose using locality-sensitive hashing (hereafter LSH) as an approximation to the single-linkage method. Under LSH, points are hashed into buckets such that the distance from a given point p need only be computed to the subset of points with which it shares a bucket.

In this paper, we implement locality-sensitive hashing as a linkage method for agglomerative hierarchical clustering, optimize the code with an emphasis on reducing the time complexity of the creation of the hash tables, and apply the algorithm to a variety of synthetic and real-world data sets.

**Keywords**: agglomerative hierarchical clustering, locality-sensitive hashing, single-linkage, nearest neighbor search, dendrogram similarity, cophenetic correlation coefficient

**Github link**: https://github.com/lisalebovici/LSHLinkClustering

## Background

With the growing ease in which data can be acquired in the modern age, techniques that can accurately represent this data are becoming increasingly important. In particular, unsupervised learning - that is, data which has no "ground truth" representation - finds itself at the center of many fields ranging from consumer marketing to computer vision to genetics. At its heart, the goal of unsupervised learning is pattern recognition. To this end, algorithms such as k-means and agglomerative hierarchical clustering have been sufficiently effective until now; but as the size of data continues to grow, scalability issues are becoming a bigger barrier to analysis and understanding.

Koga et al.'s *Fast Agglomerative Hierarchical Clustering Algorithm Using Locality-Sensitive Hashing* provides a faster approximation to single-linkage hierarchical agglomerative clustering for large data, which has a run time complexity of $O(n^2)$. The single-linkage method requires that the distances from every point to all other points not in its cluster are calculated on every iteration (of which there are $n-1$ iterations in a size $n$ data set), which becomes prohibitively expensive. In contrast, the primary advantage of LSH is that it reduces the number of distances that need to be computed as well as the the number of iterations that need to run, resulting in a linear run time complexity $O(nB)$ (where $B$ is the maximum number of points in a single hash table).

However, one consequence of the gain in run time is that LSH results in a coarser approximation of the data than single-linkage; while single-linkage necessarily creates clusters ranging from size $n$ to size $1$, LSH yields far fewer iterations of clusters. This granularity can be controlled by a parameter within the algorithm, but in general it is not expected to reproduce the single-linkage method exactly. Nonetheless, when granularity is not a primary concern, the improvements in terms of efficiency make it a worthwhile alternative.

We will proceed by describing the implementation of LSH for hierarchical clustering.

## Description of Algorithm

Some important notation will first be defined:

- $n$: number of rows in data
- $d$: dimension of data
- $C$: least integer greater than the maximal coordinate value in the data
- $\ell$: number of hash functions
- $k$: number of sampled bits from a hashed value
- $r$: minimal distance between points required to merge clusters
- $A$: increase ratio of r on each iteration

LSH works in two primary phases: first, by creating a series of hash tables in which the data points are placed, and second, by computing the distances between a point and its "similar" points to determine which clusters should be merged.

**Phase 1: Generation of hash tables**. Suppose we have a d-dimensional point $x$. A unary function is applied to each coordinate value of $x$ such that $x$ 1s are followed by $C-x$ 0s. The sequence of 1s and 0s for all coordinate values of $x$ are then joined to form the hashed point of $x$. Some number $k$ bits are then sampled without replacement from the hashed point. For example, if $x$ is the point $(2, 1, 3)$ and $C = 4$, the hashed point would be $\underbrace{1100}_{2}\underbrace{1000}_{1}\underbrace{1110}_{3}$. If $k = 5$ and the indices $I = {5, 9, 2, 11, 8}$ are randomly sampled, then our resulting value would be $11110$. $x$ would thus be placed into the hash table for $11110$.

This hash function is applied to all points, and a point is added to the corresponding hash table if no other point in its cluster is already present (that is to say, hash tables, are uniqued by cluster). Another set of $k$ indices $I$ is then randomly sampled and the resulting hash function is applied to all of the points. This procedure is repeated $\ell$ times.

The intuition behind this step is as follows: a point $s$ that is very similar to $x$ is likely to have a similar hash sequence to $x$ and therefore appear in many of the same hash tables as $x$. However, for any given hash, there is no guarantee; for example, for $s = (1, 1, 3)$, the hashed point would be $\underbrace{1000}_{1}\underbrace{1000}_{1}\underbrace{1110}_{3}$ and the sampled value would be $11010$. In this cae, $s$ would not be in the same value as $x$ above. However, by applying $\ell$ hash functions to the data, we have some certainty that if $s$ is really close to $x$, it will share at least one hash table.

**Phase 2: Nearest neighbor search**. For each point $x$, we find all of the points that share at least one hash table with $x$ and are not currently in the same cluster as $x$. These are the similar points which are candidates to have their clusters merged with $x$'s cluster. The distances between $x$ and the similar points are computed, and for all points $p$ for which the euclidean distance between $x$ and $p$ is less than $r$, $p$'s clusters are merged with $x$'s cluster.

If there is more than one cluster remaining after the merges, the values for $r$ and $k$ are updated and then phases 1 and 2 are repeated. $r$ is increased (we now consider points that are a slightly further distance away than previously to merge further-away clusters) and $k$ is decreased (so that the hashed values become shorter and therefore more common). This continues until all points are in the same cluster, at which point the algorithm terminates.

#### LSH-Link Algorithm

> **Input:** Starting values for $\ell$, $k$, and $A$

> **Initialize**:

> > **if** $n < 500$

> > > sample $M = \{\sqrt{n}$ points from data}

> > > $r = \text{min dist}(p, q)$, where $p, q \in M$

> > **else**

> > > $r = \frac{d * C}{2 * (k + d)}\sqrt{d}$

> **while** num_clusters > 1:

> > **for** $i = 1,..,\ell$:

> > > $unary_C(x) = \underbrace{11...11}_{x}\underbrace{00...00}_{C-x}$

> > > sample $k$ bits from $unary_C(x)$

> > > **if** $x$'s cluster is not in hash table:

> > > > add $x$ to hash table

> > > repeat for $n$ points

> > **for** $p = 1,..,n$:

> > > S = {set of points that share at least one hash table with $p$}

> > > Q = {$q \in S$ s.t. $\text{dist}(p, q) < r$}

> > > merge $Q$'s clusters with $p$'s cluster

> > **if** num_clusters > 1:

> > > $r = A*r$

> > > $k = \frac{d * C}{2 * r}\sqrt{d}$

## Appendices

### Appendix A

Below is the initial version of the LSH code, with run time profiling for `build_hash_tables()` and `LSHLink()`. Note that the vast majority of the run time is spent in build_hash_tables(); the code for nearest neighbor search and merging clusters is relatively trivial. The profiling shows that of the 157 seconds spent running `LSHLink()`, 151 seconds were in `build_hash_tables()`. As such, nearly all of the optimization efforts were spent in speeding up the creation of the hash tables.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn import datasets
from scipy.spatial.distance import pdist
from functools import reduce
import datetime
import pstats

def data_extend(data, k):
    r, c = data.shape
    data_extend = (reduce(lambda x, y: np.vstack((x, y)),
                          map(lambda x: data, range(k))) +
                  np.random.randn(r*c*k).reshape(r*k, c).round(1))
    return(data_extend)

def hac(k, data):
    n = data.shape[0]
    
    # start with each point in its own cluster
    clusters = np.arange(n)
    unique_clusters = len(np.unique(clusters))
    
    while unique_clusters > k:
        min_distances = np.zeros(n)
        min_points = np.zeros(n).astype('int')

        # for each point, find min distance to point not in cluster
        for i in range(n):
            point = data[i,]
            point_cluster = clusters[i]
            distances = np.linalg.norm(point - data, axis = 1)
            diff_cluster_points = np.where(clusters != point_cluster)[0]

            min_points[i] = diff_cluster_points[
                np.argmin(distances[diff_cluster_points])]
            min_distances[i] = distances[min_points[i]]

        # merge clusters of the two closest points
        point1_idx = np.argmin(min_distances)
        point1 = data[point1_idx,]
        point2_idx = min_points[point1_idx]
        point2 = data[point2_idx,]

        point2_cluster = clusters[point2_idx]
        clusters[
            np.where(clusters == point2_cluster)[0]
        ] = clusters[point1_idx]

        # update number of clusters
        unique_clusters = len(np.unique(clusters))
    
    return clusters

def unary(x, C):
    nearest_x = int(np.round(x))
    return((np.r_[np.ones(nearest_x),
                  np.zeros(C-nearest_x)]))

def lsh_hash(point, C):
    res = np.concatenate(list(map(lambda x: unary(x, C), point)))
    return(res)

def get_points_in_cluster(idx, clusters, data):
    point_cluster = clusters[idx]
    same_cluster_points_idx = np.where(
        clusters == point_cluster
    )[0]
    same_cluster_points = set(
        map(tuple, data[same_cluster_points_idx, :])
    )
    return same_cluster_points

def get_point_indices(data, points):
    indices = np.where((data == points[:,None]).all(-1))[1]
    return indices

def build_hash_tables(C, d, l, k, data, clusters):
    vals = np.arange(C*d)
    n = data.shape[0]
    hash_tables = defaultdict(set)
    hash_tables_reversed = defaultdict(set)

    for i in range(l):
        I = np.random.choice(vals, k, replace = False)

        for j in range(n):
            # for every point, generate hashed point
            # and sample k bits
            p = data[j]
            hashed_point = lsh_hash(p, C)[I]
            
            # check if any other points in p's cluster are
            # already in this hash table
            # and only add point to hash table if no other
            # points from its cluster are there
            bucket = hash_tables[tuple(hashed_point)]
            cluster_points = get_points_in_cluster(j, clusters, data)

            if not cluster_points.intersection(bucket):
                hash_tables[tuple(hashed_point)].add(tuple(p))
                hash_tables_reversed[tuple(p)].add(tuple(hashed_point))
    
    return hash_tables, hash_tables_reversed

def LSHLink(data, A, l, k, C = None, cutoff = 1):
    # set default value for C if none is provided
    if not C:
        C = int(np.ceil(np.max(data))) + 1
    
    # initializations
    n, d = data.shape
    clusters = np.arange(n)
    unique_clusters = len(np.unique(clusters))
    num = n - 1
    Z = np.zeros((n - 1, 4))
    
    # calculate r depending on n, either:
    # 1. min dist from a random sample of sqrt(n) points
    # 2. formula below
    np.random.seed(12)
    n_samp = int(np.ceil(np.sqrt(n)))
    samples = data[np.random.choice(
        n, size = n_samp, replace = False), :]
    
    if n < 500:
        r = np.min(pdist(samples, 'euclidean'))
    else:
        r = (d * C * np.sqrt(d)) / (2 * (k + d))
    
    np.random.seed(6)
    while unique_clusters > cutoff:
        # STEP 1: Generation of hash tables
        hash_tables, hash_tables_reversed = build_hash_tables(
            C, d, l, k, data, clusters)

        # STEP 2: Nearest neighbor search for p
        for i in range(n):
            # get all of those hash tables that contain point p
            p = data[i]
            p_hashes = hash_tables_reversed[tuple(p)]

            # only proceed if p is in at least one hash table
            if hash_tables_reversed[tuple(p)]:

                # find all "similar points" to p: points that
                # share at least one hash table with p, and are
                # not in the same cluster as p
                similar_points = reduce(
                    lambda x, y: x.union(y),
                    map(lambda x: hash_tables[x], p_hashes)
                    ).difference(
                    get_points_in_cluster(i, clusters, data)
                )
                similar_points = np.array(list(similar_points))

                # STEP 3: Connect pairs of clusters within certain
                # distance of p; only proceed if p has any similar points
                if similar_points.size:

                    # find similar points q s.t. dist(p, q) < r
                    # the clusters containing these points will
                    # be merged with p's cluster
                    points_to_merge = similar_points[
                        np.where(np.linalg.norm(
                            p - similar_points, axis = 1
                        ) < r)[0]
                    ]

                    # identify which clusters contain points_to_merge
                    clusters_to_merge = clusters[np.where(
                        (iris == points_to_merge[:,None]).all(-1)
                    )[1]]

                    # update cluster labels
                    clusters[np.where(
                        np.in1d(clusters,clusters_to_merge)
                    )[0]] = clusters[i]

        # STEP 4: update parameters and continue until
        # unique_clusters == cutoff
        unique_clusters = len(np.unique(clusters))

        #increase r and decrease k
        r *= A
        k = int(np.round((d * C * np.sqrt(d)) / (2 * r)))

    return(clusters)

In [3]:
iris = datasets.load_iris().data
iris = data_extend(iris, 10) * 10
iris += np.abs(np.min(iris))

l = 10
k = 100
n, d = iris.shape
C = int(np.ceil(np.max(iris))) + 1
clusters = np.arange(n)

In [4]:
%timeit -r1 hac(1, iris)

2min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%timeit -r1 LSHLink(iris, A = 1.4, l = 10, k = 100)

2min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%prun -q -D LSHLink.prof LSHLink(iris, A = 1.4, l = 10, k = 100)

 
*** Profile stats marshalled to file 'LSHLink.prof'. 


In [15]:
p = pstats.Stats('LSHLink.prof')
p.sort_stats('time', 'cumulative').print_stats(
    'build_hash_tables')
p.sort_stats('time', 'cumulative').print_stats(
    'LSHLink')
pass

Mon Apr 30 14:18:31 2018    LSHLink.prof

         26039956 function calls in 157.957 seconds

   Ordered by: internal time, cumulative time
   List reduced from 60 to 1 due to restriction <'build_hash_tables'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7   11.719    1.674  151.201   21.600 <ipython-input-1-b260ed9440e0>:68(build_hash_tables)


Mon Apr 30 14:18:31 2018    LSHLink.prof

         26039956 function calls in 157.957 seconds

   Ordered by: internal time, cumulative time
   List reduced from 60 to 1 due to restriction <'LSHLink'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.689    0.689  157.956  157.956 <ipython-input-1-b260ed9440e0>:93(LSHLink)




### Appendix B

The primary optimization in the second version of the code was caching the hash points using `lru_cache()` from `functools`. By storing the hashed unary values and point representations, the `build_hash_tables()` only had to compute each point's hash value on the initial run, rather than on each iteration; this was possible since a point's unary representation remains the same for a given C value, and C does not change throughout the course of the code.

The other optimization was to introduce more control logic into the nearest neighbor search. Specifically, additional "if" statements were added, such that if no points were found within distance $r$ of point $p$, that iteration of the for loop terminates rather than continuing on to attempt to find clusters that can be merged.

These changes reduced the run time from approximately 2 minutes 20 seconds to about 20 seconds. The code is below.

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn import datasets
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
from functools import reduce, lru_cache
import datetime
import pickle

def data_extend(data, k):
    r, c = data.shape
    data_extend = (reduce(lambda x, y: np.vstack((x, y)),
                          map(lambda x: data, range(k))) +
                  np.random.randn(r*c*k).reshape(r*k, c).round(1))
    return(data_extend)

def hac(k, data):
    n = data.shape[0]
    
    # start with each point in its own cluster
    clusters = np.arange(n)
    unique_clusters = len(np.unique(clusters))
    
    while unique_clusters > k:
        min_distances = np.zeros(n)
        min_points = np.zeros(n).astype('int')

        # for each point, find min distance to point not in cluster
        for i in range(n):
            point = data[i,]
            point_cluster = clusters[i]
            distances = np.linalg.norm(point - data, axis = 1)
            diff_cluster_points = np.where(
                clusters != point_cluster)[0]

            min_points[i] = diff_cluster_points[
                np.argmin(distances[diff_cluster_points])]
            min_distances[i] = distances[min_points[i]]

        # merge clusters of the two closest points
        point1_idx = np.argmin(min_distances)
        point1 = data[point1_idx,]
        point2_idx = min_points[point1_idx]
        point2 = data[point2_idx,]

        point2_cluster = clusters[point2_idx]
        clusters[np.where(
            clusters == point2_cluster
        )[0]] = clusters[point1_idx]

        # update number of clusters
        unique_clusters = len(np.unique(clusters))
    
    return clusters

@lru_cache(maxsize=None)
def unary(x, C):
    nearest_x = int(np.round(x))
    return(np.r_[np.ones(nearest_x), np.zeros(C-nearest_x)])

@lru_cache(maxsize=None)
def lsh_hash(point, C):
    point = np.array(point)
    res = np.concatenate(list(map(lambda x: unary(x, C), point)))
    return(res)

@lru_cache(maxsize=None)
def get_points_in_cluster(idx, clusters, data):
    clusters = np.array(clusters)
    data = pickle.loads(data)
    point_cluster = clusters[idx]
    same_cluster_points_idx = np.where(
        clusters == point_cluster)[0]
    same_cluster_points = set(
        map(tuple, data[same_cluster_points_idx, :])
    )
    return(same_cluster_points)

@lru_cache(maxsize=None)
def get_point_indices(data, points):
    data = pickle.loads(data)
    points = pickle.loads(points)
    indices = np.where((data == points[:,None]).all(-1))[1]
    return(indices)

def build_hash_tables(C, d, l, k, data, clusters):
    vals = np.arange(C*d)
    n = data.shape[0]
    hash_tables = defaultdict(set)
    hash_tables_reversed = defaultdict(set)

    for i in range(l):
        I = np.random.choice(vals, k, replace = False)

        for j in range(n):
            # for every point, generate hashed point
            # and sample k bits
            p = data[j]
            hashed_point = lsh_hash(tuple(p), C)[I]
            
            # check if any other points in p's cluster are
            # already in this hash table and only add point to
            # hash table if no other points from its cluster are there
            bucket = hash_tables[tuple(hashed_point)]
            cluster_points = get_points_in_cluster(
                j, tuple(clusters), pickle.dumps(data)
            )

            if not cluster_points.intersection(bucket):
                hash_tables[tuple(hashed_point)].add(tuple(p))
                hash_tables_reversed[tuple(p)].add(tuple(hashed_point))
    
    return hash_tables, hash_tables_reversed

def LSHLink(data, A, l, k, C = None, cutoff = 1, dendrogram = False):
    # set default value for C if none is provided
    if not C:
        C = int(np.ceil(np.max(data))) + 1
    
    if dendrogram and cutoff != 1:
        raise Exception(
            'Dendrogram requires a full hierarchy; set cutoff to 1'
        )
    
    # initializations
    n, d = data.shape
    clusters = np.arange(n)
    unique_clusters = len(np.unique(clusters))
    num = n - 1
    Z = np.zeros((n - 1, 4))
    
    # calculate r depending on n, either:
    # 1. min dist from a random sample of sqrt(n) points
    # 2. formula below
    np.random.seed(12)
    n_samp = int(np.ceil(np.sqrt(n)))
    samples = data[
        np.random.choice(n, size = n_samp, replace = False), :]
    
    if n < 500:
        r = np.min(pdist(samples, 'euclidean'))
    else:
        r = (d * C * np.sqrt(d)) / (2 * (k + d))
    
    np.random.seed(6)

    while unique_clusters > cutoff:
        # STEP 1: Generation of hash tables
        hash_tables, hash_tables_reversed = build_hash_tables(
            C, d, l, k, data, clusters)

        # STEP 2: Nearest neighbor search for p
        for i in range(n):
            # get all of those hash tables that contain point p
            p = data[i]
            p_hashes = hash_tables_reversed[tuple(p)]

            # only proceed if p is in at least one hash table
            if hash_tables_reversed[tuple(p)]:

                # find all "similar points" to p: points that share
                # at least one hash table with p, and are not in the
                # same cluster as p
                similar_points = reduce(
                    lambda x, y: x.union(y),
                    map(lambda x: hash_tables[x], p_hashes)
                    ).difference(
                    get_points_in_cluster(i, tuple(clusters),
                                          pickle.dumps(data))
                )
                similar_points = np.array(list(similar_points))

                # STEP 3: Connect pairs of clusters within certain
                # distance of p
                # only proceed if p has any similar points
                if similar_points.size:

                    # find similar points q s.t. dist(p, q) < r
                    # the clusters containing these points will be
                    # merged with p's cluster
                    points_to_merge = similar_points[
                        np.where(
                            np.linalg.norm(p - similar_points, axis = 1) < r
                        )[0]
                    ]

                    # only proceed if p has similar points within distance r
                    if points_to_merge.size:
                        # identify which clusters contain points_to_merge
                        point_indices = get_point_indices(
                            pickle.dumps(data),
                            pickle.dumps(points_to_merge)
                        )
                        clusters_to_merge = list(
                            np.unique(clusters[point_indices])
                        )
                        
                        # update cluster labels
                        # if dendrogram = False, we can use a simpler method
                        if not dendrogram:
                            clusters[
                                np.where(np.in1d(clusters, clusters_to_merge)
                                        )[0]] = clusters[i]
                        
                        else:
                            clusters_to_merge.append(clusters[i])
                            
                            for j in range(len(clusters_to_merge) - 1):
                                clusterA = clusters_to_merge[j]
                                clusterB = clusters_to_merge[j+1]
                                num += 1
                                clusters[np.where(
                                    np.in1d(clusters, [clusterA, clusterB])
                                )[0]] = num

                                Z[num - n, :] = np.array(
                                    [clusterA,
                                     clusterB,
                                     r,
                                     len(np.where(np.in1d(clusters, num))[0])]
                                )
                                clusters_to_merge[j:j+2] = 2 * [num]

        # STEP 4: update parameters and continue until
        # unique_clusters == cutoff
        unique_clusters = len(np.unique(clusters))

        #increase r and decrease k
        r *= A
        k = int(np.round((d * C * np.sqrt(d)) / (2 * r)))

    if not dendrogram:
        return(clusters)
    
    else:
        return(clusters, Z)

In [17]:
iris = datasets.load_iris().data
iris = data_extend(iris, 10) * 10
iris += np.abs(np.min(iris))

l = 10
k = 100
n, d = iris.shape
C = int(np.ceil(np.max(iris))) + 1
clusters = np.arange(n)

In [18]:
%timeit -r1 LSHLink(iris, A = 1.4, l = 10, k = 100)

21.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
